In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import pipeline as pl

OUTCOME_VAR = 'SeriousDlqin2yrs'
INDEX_COL = 0

%matplotlib inline

# Read in data and generate summary statistics

In [2]:
df = pl.read_data('cs-training.csv', INDEX_COL)

In [3]:
pl.explore_data(df)

### Summary Statistics ###
SeriousDlqin2yrs
count    150000.000000
mean          0.066840
std           0.249746
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: SeriousDlqin2yrs, dtype: float64
Null values: 0

RevolvingUtilizationOfUnsecuredLines
count    150000.000000
mean          6.048438
std         249.755371
min           0.000000
25%           0.029867
50%           0.154181
75%           0.559046
max       50708.000000
Name: RevolvingUtilizationOfUnsecuredLines, dtype: float64
Null values: 0

age
count    150000.000000
mean         52.295207
std          14.771866
min           0.000000
25%          41.000000
50%          52.000000
75%          63.000000
max         109.000000
Name: age, dtype: float64
Null values: 0

NumberOfTime30-59DaysPastDueNotWorse
count    150000.000000
mean          0.421033
std           4.192781
min           0.000000
25%           0.000000
50%           0.000000
75%           0.

# Split data into test-train sets

In [4]:
cols = list(df.columns)
features = [x for x in cols if x != OUTCOME_VAR]

In [5]:
X_train, X_test, y_train, y_test = pl.split_train_test(df, features, OUTCOME_VAR, .33)

# Fill null values with means


In [6]:
v = pl.fill_train_na_mean(X_train)

In [7]:
pl.fill_test_na_mean(X_test,v)

# Generate histograms

In [8]:
for c in features:
    pl.make_hist(X_train, c, 10)

Plotting RevolvingUtilizationOfUnsecuredLines
Plotting age
Plotting NumberOfTime30-59DaysPastDueNotWorse
Plotting DebtRatio
Plotting MonthlyIncome
Plotting NumberOfOpenCreditLinesAndLoans
Plotting NumberOfTimes90DaysLate
Plotting NumberRealEstateLoansOrLines
Plotting NumberOfTime60-89DaysPastDueNotWorse
Plotting NumberOfDependents


# Fit and test a bunch of classifiers and parameters

In [9]:
clfs,grid = pl.define_clfs_params()

In [ ]:
clfs_to_try = ['LR', 'DT', 'RF', 'SVM', 'KNN', 'AB']

In [ ]:
results = pl.try_models(clfs_to_try,clfs,grid, X_train, y_train, X_test, y_test)

# Print out a metrics report

In [ ]:
pl.print_report(results)

# Pull out best-performing model

In [ ]:
best_clf = results['DT']['Best Model']

# Read in and fill test data

In [ ]:
final_df = pl.read_data('cs-test.csv', INDEX_COL)

In [ ]:
pl.fill_test_na_mean(final_df,v)

# Predict outcomes for test data

In [ ]:
pl.predict(final_df, features, best_clf)